In [1]:
import numpy as np
import george
from george import kernels
from scipy.optimize import minimize
from time import time
import argparse
import pickle
import matplotlib.pyplot as plt

In [4]:
conv=np.load("./etgrid/3917_cobylaconv_convergence.npy")
conv=np.reshape(conv,[1000,16,16])

In [40]:
param_names = ["Tstar","logL_star","logM_disk","logR_disk","h_0","logR_in",\
          "log gamma","bix","log logM_env","logR_env","f_cav","ksi","loga_max","p","biy"]

In [24]:
print(conv[0][0])
print(conv[999][0])

[ 6.33043185 18.42068074  0.          0.         -0.4462871  -5.05145729
 -1.38629436 -2.40794561 -2.40794561 -3.21887582 -3.21887582 -2.77258872
 -2.77258872  1.38629436  2.19722458  0.81093022]
[ 5.66088825 19.42197053 -0.17330798 -0.14311232 -0.60810144 -5.26589079
 -1.60744544 -3.3282232  -3.27084462 -2.84171419 -3.67315459 -3.14019201
 -3.15567952  1.17473226  2.3055555   0.79603016]


In [ ]:
np.sqrt(np.exp(gp.get_parameter_vector()))

In [30]:
np.set_printoptions(suppress=True)
z=np.sqrt(np.exp(conv[0][0]))
n=np.sqrt(np.exp(conv[999][0]))

In [41]:
for i in range(len(param_names)):
    print(param_names[i], np.round(z[i+1],3), np.round(n[i+1],3))

Tstar 10000.0 16497.849
logL_star 1.0 0.917
logM_disk 1.0 0.931
logR_disk 0.8 0.738
h_0 0.08 0.072
logR_in 0.5 0.448
log gamma 0.3 0.189
bix 0.3 0.195
log logM_env 0.2 0.242
logR_env 0.2 0.159
f_cav 0.25 0.208
ksi 0.25 0.206
loga_max 2.0 1.799
p 3.0 3.167
biy 1.5 1.489


In [32]:
len(param_names)

15